**Notation**:
- Superscript $[l]$ denotes a quantity associated with the $l^{th}$ layer. 
    - Example: $a^{[L]}$ is the $L^{th}$ layer activation. $W^{[L]}$ and $b^{[L]}$ are the $L^{th}$ layer parameters.
- Superscript $(i)$ denotes a quantity associated with the $i^{th}$ example. 
    - Example: $x^{(i)}$ is the $i^{th}$ training example.
- Lowerscript $i$ denotes the $i^{th}$ entry of a vector.
    - Example: $a^{[l]}_i$ denotes the $i^{th}$ entry of the $l^{th}$ layer's activations).

In [69]:
import time
import numpy as np
import h5py
import matplotlib.pyplot as plt
import scipy
from testCases import *
from PIL import Image
from scipy import ndimage
from dnn_utils import sigmoid, sigmoid_backward, relu, relu_backward
from ld_utils import load_data

%matplotlib inline
plt.rcParams['figure.figsize'] = (5.0, 4.0) # default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


#### Overview of steps ####

- Intialize parameters for a two-layer network and for an $L$-layer neural network.
- Implement the forward propagation module (purple)
    - Compute LINEAR part of a layer's forward propagation step (resulting in $Z^{[l]}$)
    - Get the ACTIVATION function (relu / sigmoid)
    - Combine LINEAR part with ACTIVATION function into a new [LINEAR -> ACTIVATION] forward function.
    - Stack the [LINEAR -> RELU] forward function L-1 times (for layers 1 through L-1) and add a [LINEAR -> SIGMOID] at the end (for final layer $L$) which gives the L_model_forward function.
- Compute the loss
- Implement the backward propagation module (red)
    - Compute the LINEAR part of a layer's backward propagation step.
    - Get the gradient of the ACTIVATION function  (relu_backward / sigmoid_backward)
    - Combine the two parts into a new [LINEAR -> ACTIVATION] backward function.
    - Stack [LINEAR -> RELU] backward L-1 times and add [LINEAR -> SIGMOID] backward in a new L_model_backward function.
- Update the parameters


<img src="images/final outline.png" style="width:800px;height:500px;">

**For every forward function there is a backward function so store values into cache**

In [70]:
# helper function to intialize parameters for a two layer model
def initialize_parameters(n_x, n_h, n_y):
    
    W1 = np.random.randn(n_h, n_x) * 0.01
    b1 = np.zeros((n_h, 1))
    W2 = np.random.randn(n_y, n_h) * 0.01
    b2 = np.zeros((n_y, 1))
    
    assert(W1.shape == (n_h, n_x))
    assert(b1.shape == (n_h, 1))
    assert(W2.shape == (n_y, n_h))
    assert(b2.shape == (n_y, 1))
    
    parameters = {'W1' : W1,
                  'b1' : b1,
                  'W2' : W2,
                  'b2' : b2}
    
    return parameters

$n^{[l]}$ is the number of units in layer $l$. For example, if the size of output $X$ is $(12288, 209)$ with $m = 209$ then :


The initialization for a deeper L-layer neural network is more complicated because there are many more weight matrices and bias vectors. When completing the `initialize_parameters_deep`, you should make sure that your dimensions match between each layer. Recall that $n^{[l]}$ is the number of units in layer $l$. Thus for example if the size of our input $X$ is $(12288, 209)$ (with $m=209$ examples) then:

<table style="width:100%">
    <tr>
        <td>  </td> 
        <td> **Shape of W** </td> 
        <td> **Shape of b**  </td> 
        <td> **Activation** </td>
        <td> **Shape of Activation** </td> 
    <tr>
    <tr>
        <td> **Layer 1** </td> 
        <td> $(n^{[1]},12288)$ </td> 
        <td> $(n^{[1]},1)$ </td> 
        <td> $Z^{[1]} = W^{[1]}  X + b^{[1]} $ </td>     
        <td> $(n^{[1]},209)$ </td> 
    <tr>
    <tr>
        <td> **Layer 2** </td> 
        <td> $(n^{[2]}, n^{[1]})$  </td> 
        <td> $(n^{[2]},1)$ </td> 
        <td>$Z^{[2]} = W^{[2]} A^{[1]} + b^{[2]}$ </td> 
        <td> $(n^{[2]}, 209)$ </td> 
    <tr>
    <tr>
        <td> $\vdots$ </td> 
        <td> $\vdots$  </td> 
        <td> $\vdots$  </td> 
        <td> $\vdots$</td> 
        <td> $\vdots$  </td> 
    <tr>
    <tr>
        <td> **Layer L-1** </td> 
        <td> $(n^{[L-1]}, n^{[L-2]})$ </td> 
        <td> $(n^{[L-1]}, 1)$  </td> 
        <td>$Z^{[L-1]} =  W^{[L-1]} A^{[L-2]} + b^{[L-1]}$ </td> 
        <td> $(n^{[L-1]}, 209)$ </td> 
    <tr>
    <tr>
        <td> **Layer L** </td> 
        <td> $(n^{[L]}, n^{[L-1]})$ </td> 
        <td> $(n^{[L]}, 1)$ </td>
        <td> $Z^{[L]} =  W^{[L]} A^{[L-1]} + b^{[L]}$</td>
        <td> $(n^{[L]}, 209)$  </td> 
    <tr>

</table>


In [71]:
# helper function for parameter initialization of an L-layer Neural Network
def initialize_parameters_deep(layer_dims):
    
    parameters = {}
    L = len(layer_dims)  # number of layers in the network
    for l in range(1, L):
        parameters['W' + str(l)] = np.random.randn(layer_dims[l], layer_dims[l - 1]) * 0.01
        parameters['b' + str(l)] = np.zeros((layer_dims[l], 1))
        
        assert(parameters['W' + str(l)].shape == (layer_dims[l], layer_dims[l - 1]))
        assert(parameters['b' + str(l)].shape == (layer_dims[l], 1))
        
    return parameters

In [72]:
# implementing linear part of the forward model
def linear_forward(A, W, b):
    
    Z = np.dot(W, A) + b
    assert(Z.shape == (W.shape[0], A.shape[1]))
    cache = (A, W, b)
    
    return Z, cache

In [73]:
# implementing the forward propagation of the LINEAR->ACTIVATION layer: A = g(Z)
# performs the LINEAR forward step followed by the ACTIVATION forward step
def linear_activation_forward(A_prev, W, b, activation):
    
    if activation == 'sigmoid':
        Z, linear_cache = linear_forward(A_prev, W, b)
        A, activation_cache = sigmoid(Z)
        
    elif activation == 'relu':
        Z, linear_cache = linear_forward(A_prev, W, b)
        A, activation_cache = relu(Z)
        
    assert(A.shape == (W.shape[0], A_prev.shape[1]))
    cache = (linear_cache, activation_cache)
    
    return A, cache

variable `AL` denotes $A^{[L]} = \sigma(Z^{[L]}) = \sigma(W^{[L]} A^{[L-1]} + b^{[L]})$. (This is sometimes also called `Yhat`, i.e., $\hat{Y}$.)

In [74]:
# implementing the forward propagation model
def L_model_forward(X, parameters):
    
    caches = []
    A = X
    L = len(parameters) // 2
    
    for l in range(1, L):
        
        A_prev = A
        A, cache = linear_activation_forward(A_prev, parameters['W' + str(l)], parameters['b' + str(l)], activation='relu')
        caches.append(cache)
        
    AL, cache = linear_activation_forward(A, parameters['W' + str(l)], parameters['b' + str(l)], activation='sigmoid')
    caches.append(cache)
    
    assert(AL.shape == (1, X.shape[1]))
    
    return AL, caches

In [75]:
# computing the cross entropy cost J
def compute_cost(AL, Y):
    
    m = Y.shape[1]
    cost = - np.sum((np.multiply(np.log(AL), Y) + np.multiply(np.log(1 - AL), (1 - Y)))) / m
    
    cost = np.squeeze(cost)
    assert(cost.shape == ())
    
    return cost

<img src="images/backprop_kiank.png" style="width:650px;height:250px;">

<caption><center>Forward and Backward propagation for *LINEAR->RELU->LINEAR->SIGMOID* </center></caption>

Backward propagation steps similar to forward propagation :
- LINEAR backward
- LINEAR -> ACTIVATION backward where activation computes the derivative of either the Relu or the sigmoid activation
- [LINEAR -> RELU] $\times$ (L-1) -> LINEAR -> SIGMOID backward (complete model)

#### Linear backward ####

For layer $l$, the linear part is: $Z^{[l]} = W^{[l]} A^{[l-1]} + b^{[l]}$ (followed by an activation).

derivative $dZ^{[l]} = \frac{\partial \mathcal{L} }{\partial Z^{[l]}}$.

<img src="images/linearback_kiank.png" style="width:250px;height:300px;">
<caption><center> **Figure 4** </center></caption>

$$ dW^{[l]} = \frac{\partial \mathcal{J} }{\partial W^{[l]}} = \frac{1}{m} dZ^{[l]} A^{[l-1] T} \tag{8}$$
$$ db^{[l]} = \frac{\partial \mathcal{J} }{\partial b^{[l]}} = \frac{1}{m} \sum_{i = 1}^{m} dZ^{[l](i)}\tag{9}$$
$$ dA^{[l-1]} = \frac{\partial \mathcal{L} }{\partial A^{[l-1]}} = W^{[l] T} dZ^{[l]} \tag{10}$$


In [76]:
# helper funtion to compute the linear part of the backward propagation step
def linear_backward(dZ, cache):
    
    A_prev, W, b = cache
    m = A_prev.shape[1]
    
    dW = np.dot(dZ, A_prev.T) / m
    db = np.sum(dZ, axis = 1, keepdims = True) / m
    dA_prev = np.dot(W.T, dZ)
    
    assert(dW.shape == W.shape)
    assert(db.shape == b.shape)
    assert(dA_prev.shape == A_prev.shape)
    
    return dA_prev, dW, db

If $g(.)$ is the activation function, 
`sigmoid_backward` and `relu_backward` compute $$dZ^{[l]} = dA^{[l]} * g'(Z^{[l]}) \tag{11}$$

In [77]:
# backward propagation step for the LINEAR->ACTIVATION layer
# combines the linear_backward function with the backward step
def linear_activation_backward(dA, cache, activation):
    
    linear_cache, activation_cache = cache
    
    if activation == 'relu':
        dZ = relu_backward(dA, activation_cache)
        dA_prev, dW, db = linear_backward(dZ, linear_cache)
        
    elif activation == 'sigmoid':
        dZ = sigmoid_backward(dA, activation_cache)
        dA_prev, dW, db = linear_backward(dZ, linear_cache)
        
    return dA_prev, dW, db

In [1]:
# combining everything and implementing a backward model for L layers
def L_model_backward(AL, Y, caches):
    
    grads = {}
    L = len(caches)  # number of layers
    m = AL.shape[1]
    Y = Y.reshape(AL.shape)
    
    # initalizing the backpropagation
    dAL = - np.divide(Y, AL) + np.divide(1 - Y, 1 - AL)
    
    # gradients for Lth layer
    current_cache = caches[-1]
    grads['dA' + str(L-1)], grads['dW' + str(L)], grads['db' + str(L)] = linear_activation_backward(dAL, current_cache, activation='relu')
    
    # loop from l-2 to 0 for rest of the layers
    for l in reversed(range(L-1)):
        
        current_cache = caches[l]
        dA_prev_temp, dW_temp, db_temp = linear_activation_backward(grads['dA' + str(l + 1)], current_cache, activation='sigmoid')
        grads['dA' + str(l)] = dA_prev_temp
        grads['dW' + str(l)] = dW_temp
        grads['db' + str(l)] = db_temp
        
    return grads

In [2]:
def update_parameters(parameters, grads, learning_rate):
    
    L = len(paramters) //  2
    
    for l in range(L):
        parameters["W" + str(l+1)] -= learning_rate * grads["dW" + str(l + 1)]
        parameters["b" + str(l+1)] -= learning_rate * grads["db" + str(l + 1)]
        
    return parameters